In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pandas as pd
from pathlib import Path
import dateutil.parser
from NPw_aux import read_iono_data, process_ionosphere_files, read_EQ_data, process_eq
from datetime import timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [299]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''

import os, json, requests

locations = [(40, 23)]

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start=20150101&end=20150305&format=JSON"

for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    content = json.loads(response.content.decode('utf-8'))
    filename = response.headers['content-disposition'].split('filename=')[1]

    filepath = os.path.join(output, filename)
    with open(filepath, 'w') as file_object:
        json.dump(content, file_object)

In [164]:
freq = timedelta(minutes=30)
df_GNSSTEC,df_covariate, df_eq = prepare_ion_data(datapath, "GRK", freq)

In [214]:
set_log_level("ERROR")
datapath = datapath
delta = timedelta(minutes=30)
# Read SR and EQ data
freq = timedelta(minutes = 30)


In [196]:
from NPw import multivariate_df
print(len(df_covariate))

87743


In [ ]:
freq = timedelta(minutes=30)
df_GNSSTEC,df_covariate, df_eq = prepare_ion_data(datapath, "GRK", freq)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from aux_function_SR import read_data, get_eq_filtered, SR_SENSORS
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level, save, load
import logging
import pandas as pd
from datetime import datetime, timedelta
import pickle
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast
from dataclasses import dataclass, asdict
from dateutil.relativedelta import *
from NPw_aux import prepare_ion_data
import ast
from pathlib import Path
import sys
import logging

logging.disable(logging.CRITICAL)
datapath = str(Path.home()) + "/data/"

In [2]:
freq = timedelta(minutes=30)
df_GNSSTEC,df_covariate, df_eq = prepare_ion_data(datapath, "GRK", freq)

In [3]:
df_regressor = df_GNSSTEC.reset_index()
df_other = df_covariate
df_events = df_eq
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast, METRICS

forecast_length = timedelta(hours=24)
question_mark_length = timedelta(hours=24)
# Time to take into account to predict 
historic_lenght =  timedelta(days=5)
training_lenght = timedelta(days=365)


config_npw_d = {
    "forecast_length": forecast_length,
    "question_mark_length": question_mark_length,
    "freq": freq,
    "training_length": training_lenght,
    "historic_lenght": historic_lenght,
    "drop_missing": False,
    "num_hidden_layers": 10,
    "learning_rate": None,
    "d_hidden": 16,
    "yearly_seasonality": True,
    "daily_seasonality" : False,
    "weekly_seasonality" : True,
    "seasonal_mode": "multiplicative",
    "seasonal_reg": 0,
    "multivariate_season": True,
    "multivariate_trend": True,
    "verbose": False,
    "epochs": None,
    "drop_missing": False,
    "use_gpu": False,
    "event_type": "Non-Binary",
    "event_mode": "multiplicative",
    "normalization": True,
    "trend_regularization": 0,
    "trend_n_changepoint": 10,
    "sparce_AR": None,
    "loss_func": "Huber",
    "growth": "off"
}

config_npw = ConfigNPw(**config_npw_d)
ConfigEQ_d = {
    "dist_start": 1000,
    "dist_delta": 3000,
    "mag_start": 4.5,
    "mag_delta": 2,
    "filter": True,
    "drop": ["arc_cos", "arc_sin"]
}
config_events = ConfigEQ(**ConfigEQ_d)

synthetic_events = pd.read_pickle("synthetic.pkl")
#config_path = "config_002.json"
#(config_npw, config_events) = NPw.save_config(config_path)

In [4]:
NPw_o = NPw(config_npw, df_regressor,df_other, config_events, df_events, synthetic_events)

m1: 4.5 m2: 6.5 d1: 1000 d2: 4000


In [5]:
hours_offsets = [0]
event_offsets = [None,  -timedelta(hours=24)]
index = 1


# see what happen in 2019-02-01T10:00:00 Nan values at the end
start_day = datetime.fromisoformat("2018-10-01T10:00:00")

start_date = start_day + index * relativedelta(months=+1)

In [7]:
number_of_folds = 12
NPw_o.get_folds(k = number_of_folds)

In [9]:
df_forecast, df_uncer = NPw_o.process_fold(1)

2017-08-30 15:30:00


Finding best initial lr:   0%|          | 0/270 [00:00<?, ?it/s]

In [11]:
from NPw import METRICS
NPw_o.get_metrics_from_fc(df_forecast["BASE"], METRICS.CoV) #18.623972359951672 26.79848069427478  # 17.83331678074142

21.58617144495947

In [22]:
df_forecast["BASE"]


current                                        pred  \
                          duth       dyng       larm       noa1       duth   
ds                                                                           
2017-05-25 04:00:00  11.587167  11.698500  10.920167  11.913667  17.537910   
2017-05-25 04:30:00  12.578667  13.493833  12.209000  13.413667  17.963572   
2017-05-25 05:00:00  13.769667  14.931000  13.395000  14.660167  18.009300   
2017-05-25 05:30:00  14.606667  16.039500  14.410167  15.702500  18.284714   
2017-05-25 06:00:00  15.094833  16.352833  14.853000  16.029167  18.312740   
2017-05-25 06:30:00  14.933333  16.331500  14.639667  16.044833  20.506855   
2017-05-25 07:00:00  14.851667  16.704167  14.927000  16.347667  20.546715   
2017-05-25 07:30:00  15.190500  17.393167  16.057000  16.872667  20.455467   
2017-05-25 08:00:00  15.398833  17.254500  16.762333  16.546333  20.491684   
2017-05-25 08:30:00  14.689667  15.982000  15.952167  15.137500  20.590588   
2017-05-25 09:00:00  13.625667  15.178667  14.769000  14.180833  20.657768   
2017-05-25 09:30:00  12.876667  14.952833  13.894667  13.759500  19.542145   
2017-05-25 10:00:00  12.382833  14.820167  13.404833  13.486833  19.569851   
2017-05-25 10:30:00  12.466000  15.089833  13.611667  13.766833  24.611794   
2017-05-25 11:00:00  12.792667  15.389000  13.955500  13.996667  24.634720   
2017-05-25 11:30:00  12.792000  15.427167  13.873333  14.074000  22.243862   
2017-05-25 12:00:00  12.279833  14.897333  13.177000  14.051500  22.276749   
2017-05-25 12:30:00  11.627167  13.898333  12.320333  13.616000  23.169678   
2017-05-25 13:00:00  11.155667  13.347500  11.764833  13.417667  23.182270   
2017-05-25 13:30:00  11.321833  13.448333  11.743333  13.681167  22.175911   
2017-05-25 14:00:00  11.893833  13.897000  12.225500  14.136833  22.183064   
2017-05-25 14:30:00  12.568167  14.098833  12.711500  14.393333  18.168640   
2017-05-25 15:00:00  12.795833  13.746000  12.804333  14.192667  18.170252   
2017-05-25 15:30:00  13.125167  13.741000  13.127833  14.279333  18.169073   
2017-05-25 16:00:00  14.041833  14.435500  13.963333  15.027833  18.165104   
2017-05-25 16:30:00  15.244333  15.522833  15.153667  16.203833  18.176823   
2017-05-25 17:00:00  16.066833  16.371667  16.185667  17.108667  18.167294   
2017-05-25 17:30:00  16.071833  16.419000  16.461500  17.133333  18.136536   
2017-05-25 18:00:00  15.428833  15.761333  15.995000  16.484333  18.086502   
2017-05-25 18:30:00  14.594333  14.382167  15.335667  15.288333  18.068817   
2017-05-25 19:00:00  13.946500  12.872500  14.593667  14.123500  18.048492   
2017-05-25 19:30:00  13.029167  12.052667  13.768333  13.390167  18.025581   
2017-05-25 20:00:00  12.419667  11.572167  12.982333  12.703333  18.000141   
2017-05-25 20:30:00  11.739000  10.961333  11.858833  11.857833  17.972242   
2017-05-25 21:00:00  11.037667  10.101667  10.570333  10.697000  17.976990   
2017-05-25 21:30:00   9.895333   9.323000   9.164833   9.617667  17.944397   
2017-05-25 22:00:00   9.056167   8.786167   8.301667   9.076333  17.909588   
2017-05-25 22:30:00   8.658000   8.414000   7.894667   8.848167  17.872654   
2017-05-25 23:00:00   8.204333   7.922667   7.159000   8.368333  17.833694   
2017-05-25 23:30:00   7.984333   7.652500   6.825000   8.159500  17.792812   
2017-05-26 00:00:00  17.123000  16.243000  18.323667  17.821500  18.217850   
2017-05-26 00:30:00  17.078333  15.712000  18.432333  17.849167  18.173466   
2017-05-26 01:00:00  16.874667  15.184333  18.317000  17.801833  18.127508   
2017-05-26 01:30:00  16.363667  14.757000  17.697500  17.356167  18.080101   
2017-05-26 02:00:00  16.262667  14.596167  17.422333  17.061667  18.031376   
2017-05-26 02:30:00  16.981167  14.964833  17.579167  17.320000  17.981462   
2017-05-26 03:00:00  18.682333  16.551500  18.909000  18.745500  17.895466   
2017-05-26 03:30:00  20.348167  18.524000  20.625167  20.576500  17.843588   

                                            

In [270]:
current_test = NPw_o.df_test.copy()
array_bool = [current_test["ds"] > start_day]
current_test.loc[current_test["ds"] > start_day,"y"] = np.nan

NPw_o.model.predict(current_test, decompose=False, raw= True)

,ds,ID,step0,step0 9.999999999999998%,step0 90.0%,step1,step1 9.999999999999998%,step1 90.0%,step2,step2 9.999999999999998%,...,step44 90.0%,step45,step45 9.999999999999998%,step45 90.0%,step46,step46 9.999999999999998%,step46 90.0%,step47,step47 9.999999999999998%,step47 90.0%
0,2016-09-19 16:00:00,__df__,28.569645,27.717892,29.943884,28.308132,27.596943,29.665751,27.788723,27.132927,...,29.120209,25.535583,21.672941,28.742863,25.454977,21.407759,28.802835,25.218382,21.178875,28.368673
1,2016-09-19 16:00:00,__df__,27.513187,26.756083,28.803431,27.218555,26.629131,28.601614,26.644686,26.076700,...,28.295713,24.850668,21.196100,27.900164,24.726170,20.966965,27.923914,24.528130,20.715132,27.622784
2,2016-09-19 16:00:00,__df__,30.861229,29.878111,32.354370,30.628454,29.804325,32.077820,30.069328,29.330339,...,31.572954,27.720268,23.581932,31.208052,27.644871,23.238289,31.281326,27.332888,22.978613,30.663671
3,2016-09-19 16:00:00,__df__,30.686199,29.720608,32.152962,30.478512,29.661427,31.894474,29.921951,29.194557,...,31.679600,27.897221,23.917976,31.347393,27.825130,23.554096,31.369205,27.476624,23.260517,30.689922


19.100224368543607

In [77]:
new = df_forecast["BASE"]

In [79]:
old

NameError: name 'old' is not defined

In [ ]:
for n_fold in range(number_of_folds):
    try:
        NPw_o.process_fold(n_fold)
    except: #
        print("Problem in fold " + str(n_fold))
        

In [ ]:
import pyarrow.feather as feather
feather.write_feather(NPw_o.get_results()[0], 'test')


In [ ]:
8 //3

In [ ]:
df, df_MAE = NPw_o.get_results()

In [ ]:
df

In [ ]:
df_scale = df.apply(lambda x: ((x -x[0])/x[0]), axis = 1)

In [ ]:
df_scale = ((df_scale / df_scale.abs().max().max()) *100).astype(int)

In [ ]:
NPw.print_df(df)

In [ ]:
df_scale.to_csv("binary_ex.csv")

In [ ]:
print(type(df.index))
b = list()
[b.append(item[0]) for item in df.index if item not in b]
b = (pd.DataFrame(b, columns = ["dates"]).drop_duplicates())
b

In [ ]:
def filter_events(df_events, fc_dt, sort_value, low_td, high_td):
    df_events.index = df_events.index + config_npw.forecast_length // 2
    events = df_events.iloc[(df_events.index > fc_dt - low_td) & (df_events.index < fc_dt + high_td)].sort_values("pr", ascending = False).head()
    if events.empty == False:
        return events

In [ ]:
current_events = list()
for fc_dt in b["dates"]:
    current_events.append(filter_events(df_events.copy(), fc_dt, "pr", timedelta(0), timedelta(150)))

In [ ]:
synthetic_events = [x.iloc[0] for x in current_events[2:8]]

In [ ]:
pd.concat(synthetic_events,axis = 1, ignore_index=True).T.to_pickle("synthetic2.pkl")

In [ ]:
df

In [ ]:
df.to_pickle("ion_test_grk.pkl")

In [ ]:
from scipy.stats import zscore
zetascore_table=zscore(df,axis=0)
zetascore_table

In [ ]:
dates = [x[0] for x in df.keys()].unique()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(50))]

In [ ]:
df

In [ ]:
df_events.iloc[(df_events.index > df.keys()[1][0] - timedelta(0)) & (df_events.index < df.keys()[1][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[4][0] - timedelta(0)) & (df_events.index < df.keys()[4][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[6][0] - timedelta(0)) & (df_events.index < df.keys()[6][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
df_events.iloc[(df_events.index > df.keys()[10][0] - timedelta(0)) & (df_events.index < df.keys()[10][0] + timedelta(2))].sort_values("pr", ascending = False).head()

In [ ]:
len(NPw_o.list_keys_fold)

In [ ]:
(NPw_o.list_test_RMSE[0].reset_index())

In [ ]:
a = NPw_o.list_test_RMSE[0][0:3].copy()
b = NPw_o.list_test_RMSE[0][3:6].copy()
c = NPw_o.list_test_RMSE[0][3:6].copy()
a.co"]
b.columns = ["Blumns = ["A"]
c.columns = ["C"]
print(a)
print(b)
r1 = pd.concat([a,b, c], axis = 1).copy()
r2 = pd.concat([a,b, c], axis = 1).copy()
r3 = pd.concat([a,b, c], axis = 1).copy()
print(pd.concat([r1,r2,a], axis=1, keys = [1,2,"NOTHING"]))

In [ ]:
from sdv.tabular import GaussianCopula

In [ ]:
model = GaussianCopula()
model.fit(events)

In [ ]:
sample = model.sample(2000)
sample.head()
print(sample.nlargest(1, columns = ["mag"]))
print(sample.nlargest(5, columns = ["dist"]))
example = sample.nlargest(5, columns = ["dist"])
idx = example.index[[True, False, True, False, True]]

In [ ]:
for idx, ex in example.iterrows():
    print(ex)
    for e in ex.index:
        print(e)

In [ ]:
len(df_train)

In [ ]:
df_train = NPw_o.df_train
delta_t = NPw_o.config_npw.forecast_length

In [ ]:
t0 = df_train.iloc[-(NPw_o.n_forecasts)]["ds"]
t1 = t0 + delta_t

In [ ]:
series = pd.date_range(start=t0, end=t1, periods=5)
series.values

In [ ]:
df_test = df_train.copy()

In [ ]:
df_test["ds"].get_loc()

In [ ]:
print(ev.iloc[1])
print(ev.iloc[2])
ev.iloc[2] = ev.iloc[1]
print(ev.iloc[2])

In [ ]:
for index, (df_train, df_test) in enumerate(NPw_o.folds):
    if index == 10:
        print(df_test)


In [ ]:
config_fc = ConfigForecast(
    start_forecast=start_date, offset_event=None
)
metrics= NPw_o.add_forecast(config_fc
)

In [ ]:
metrics[1]

In [ ]:
NPw_o.one_step_test(model, config_fc, df_test)

In [ ]:
future = model.one_step(df_test, n_historic_predictions=24)

In [ ]:
forecast = model.predict(future, decompose=False, raw=True)
forecast_2  = forecast[forecast["ds"] == start_date].T

In [ ]:
forecast_2.columns = df_test_2.columns  + "_pred"
df_test_2.columns =  df_test_2.columns  + "_current"

In [ ]:
forecast_2 = forecast_2.drop(["ds" ,"ID"]).reset_index().drop("index", axis = 1)

In [ ]:
df_test

In [ ]:
pd.pivot(df_test, index="ds", columns="ID", values="y")[-24:]

In [ ]:
df_all = pd.concat([df_test_2.reset_index(), forecast_2], axis = 1).drop("ds", axis = 1)
df_all.columns = df_all.columns.str.split('_', expand=True)
df_all = df_all.swaplevel(axis=1) 
mse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0)
mae = df_all['current'].sub(df_all['pred']).abs().mean(axis=0)
rmse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0).pow(1/2)
results = pd.concat([mse, mae, rmse], axis = 1)
results

In [ ]:
 pd.pivot(df_test, index="ds", columns="ID", values="y")[-(self.n_forecasts):]

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix

df_test_2.apply(mean_squared_error, axis=0)
